# Top Tagging with Particle Flow Network

Let's try this on photon jet

Documentation: [Examples](https://energyflow.network/examples/), [Architectures](https://energyflow.network/docs/archs/)

In [11]:
# Make tensorflow quieter
# This takes up to a minute...
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Computing imports
import math
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve

# Useful imports
from tqdm import tqdm
import matplotlib.pyplot as plt

# Energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split, to_categorical

In [8]:
# Utility functions and whatnot
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


class_labels = ["pion", "photon", "scalar"]

In [2]:
# Make sure tensorflow can use the GPU
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# matplotlib settings
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.autolayout'] = True

## Load data

In [18]:
# ~10 sec
jets_path = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data/processed/scalar_test/all_jets_point_cloud.npz"
jets = dict(np.load(jets_path))

X, y = jets["X"], jets["y"]
Y = to_categorical(y)

print("Loaded data")

Loaded data


### Feature information

`X`: `(N, max_jet_size, 5)` 3D numpy array of floats
* `0`: $p_T$, transverse momentum
* `1`: $\eta$, angular coordinate
* `2`: $\phi$, angular coordinate
* `3`: $E$, energy
* `4`: $R$, $\sqrt{\eta^2 + \phi^2}$

`y`: `(N, 5)` 1D numpy array of ints that are one-hot encoded
* `0`: Gluon
* `1`: Light quark
* `2`: W boson
* `3`: Z boson
* `4`: Top quark


In [19]:
# ~5 sec
# Do train/val/test split
n_val = 20000
n_test = 20000

(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=n_val, test=n_test)

print("Done train/val/test split")

Done train/val/test split


In [20]:
print(X_train.shape, Y_train.shape)

(260000, 960, 4) (260000, 3)


In [21]:
for data_name, data in zip(
    ["X_train", "X_val", "X_test"],
    [X_train, X_val, X_test]
):
    print(data_name.ljust(15), convert_size(data.nbytes))

X_train         7.44 GB
X_val           585.94 MB
X_test          585.94 MB


## Build the model

In [22]:
# Hyperparameters

# Network architecture parameters
Phi_sizes = [100, 100, 100, 256]
F_sizes = [100, 100, 100, 100]

# Network training parameters
n_epochs = 1
batch_size = 500

In [23]:
# Might take a few seconds
pfn = PFN(input_dim=X.shape[-1],
          output_dim=Y.shape[-1],
          Phi_sizes=Phi_sizes,
          F_sizes=F_sizes)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, 4)]    0           []                               
                                                                                                  
 tdist_0 (TimeDistributed)      (None, None, 100)    500         ['input[0][0]']                  
                                                                                                  
 activation_18 (Activation)     (None, None, 100)    0           ['tdist_0[0][0]']                
                                                                                                  
 tdist_1 (TimeDistributed)      (None, None, 100)    10100       ['activation_18[0][0]']          
                                                                                            

In [24]:
# Train the model!
pfn.fit(X_train, Y_train,
        epochs=n_epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        verbose=1)

520/520 [==============================] - 58s 80ms/step - loss: 2843.3789 - acc: 0.3349 - val_loss: 1006.2841 - val_acc: 0.3290


## Evaluate model

In [25]:
# get predictions on test data
preds = pfn.predict(X_test, batch_size=500)

40/40 [==============================] - 1s 24ms/step


In [26]:
test_labels = np.argmax(Y_test, axis=1)
pred_labels = np.argmax(preds, axis=1)

In [27]:
mask = (test_labels == pred_labels).astype(float)
print(mask)
print(f"Test accuracy: {mask.mean()}")

[0. 0. 0. ... 1. 0. 1.]
Test accuracy: 0.33625
